### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [8]:
num_iterations = 1
empty_list = []
for i in range(850, 950):
    if i == 724:
        continue

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_e
class_probability = 0.58


######### Lightcurve features #########

amplitude = 0.049847417369495536
cusum = 0.14592012321303663
eta = 2.489180130207388
hl_amp_ratio = 1.129193752775512
kurtosis = 0.24350297622970984
n_points = 548
period = 0.29711581091961814
period_SNR = 66.12068782061432
period_log10FAP = -26.09365820042794
period_uncertainty = 3.737776017384564e-05
phase_cusum = 0.19901866103534102
phase_eta = 1.6953024362519729
phi21 = -2.7463714359242
phi31 = -5.057885872438377
quartile31 = 0.0838299727742644
r21 = 0.15111692701393606
r31 = 0.148312665048657
shapiro_w = 0.988027036190033
skewness = 0.3883221961240636
slope_per10 = -0.05558957605885449
slope_per90 = 0.04650603595579003
stetson_k = 0.7720966563027143
weighted_mean = 15.134212417721239
weighted_std = 0.06078390722750267
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.37076686638734924
cusum = 0.17081424497697814
eta = 1.80867963402304
hl_amp_ratio = 0.28570166698645294
kurtosis = -0.4977682643423078
n_points = 2007
period = 0.6134082791875636
period_SNR = 271.2651967212133
period_log10FAP = -309.1715137428492
period_uncertainty = 0.00015905142275579598
phase_cusum = 0.3482553179930043
phase_eta = 0.10048998871555981
phi21 = 3.769542112893933
phi31 = 4.565935020487472
quartile31 = 0.4469139120493306
r21 = 0.38280500315971955
r31 = 0.30314508707850546
shapiro_w = 0.8835020661354065
skewness = -0.8388308618657139
slope_per10 = -0.035197796560131375
slope_per90 = 0.034527254235703325
stetson_k = 0.6736592811225866
weighted_mean = 14.39648058322351
weighted_std = 0.30971173948628816
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.85


######### Lightcurve features #########

amplitude = 0.009250162943483528
cusum = 0.19965825984587432
eta = 3.434580672912874
hl_amp_ratio = 0.788819740769436
kurtosis = 7.575893974491686
n_points = 489
period = 0.4993185410533115
period_SNR = 10.622263031745145
period_log10FAP = -0.184939649254019
period_uncertainty = 0.00012320162128434675
phase_cusum = 0.15831226477462895
phase_eta = 3.41145049612057
phi21 = -1.935176617024775
phi31 = -4.887386015532966
quartile31 = 0.023842354864065385
r21 = 0.23295532465684002
r31 = 0.15813000661406326
shapiro_w = 0.906330406665802
skewness = 0.13667008593408045
slope_per10 = -0.23502205711047072
slope_per90 = 0.2610370218198371
stetson_k = 0.8022631648963402
weighted_mean = 14.170556631374845
weighted_std = 0.01795113931446404
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.24787588281967338
cusum = 0.12840400553980227
eta = 2.376296024420456
hl_amp_ratio = 0.47340397504849424
kurtosis = -0.9155813593834541
n_points = 1045
period = 0.6209367082749482
period_SNR = 196.02962757874084
period_log10FAP = -152.31379799160123
period_uncertainty = 0.00013594376408720965
phase_cusum = 0.28026549466840495
phase_eta = 0.28529751980009105
phi21 = 0.7257776394627358
phi31 = 1.9803790782260824
quartile31 = 0.3434374030275382
r21 = 0.4194160452508333
r31 = 0.2416769034884917
shapiro_w = 0.9319344162940979
skewness = -0.47573106513407853
slope_per10 = -0.05297247351190418
slope_per90 = 0.05644056459739275
stetson_k = 0.7658977731128277
weighted_mean = 14.995655513183715
weighted_std = 0.20409019171625697


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
